In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("recommandation").getOrCreate()

In [ ]:
df_train = spark.read.csv('',header=True,inferSchema=True)


In [ ]:
df_train.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- rating: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import format_number, lit
df_train = df_train.withColumn('rating', df_train.rating.cast('float'))

In [ ]:
df_train.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- rating: float (nullable = true)



In [ ]:
df_train.describe().show()

+-------+--------------------+--------------------+------------------+
|summary|                item|                user|            rating|
+-------+--------------------+--------------------+------------------+
|  count|               83798|               83798|             83798|
|   mean|2.0769135928283918E9|                null| 4.359018114990811|
| stddev|2.0710778913333938E9|                null|0.9940809586163019|
|    min|          0439893577|A012468118FTQAINE...|               1.0|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|               5.0|
+-------+--------------------+--------------------+------------------+



In [ ]:
df_train.head(10)

[Row(item='B008H54GVE', user='A2OIMJEGOCTQ87', rating=4.0),
 Row(item='B001NLISDG', user='A38KQZS5M1A8T8', rating=4.0),
 Row(item='B007S3S8HO', user='A1F5O1USOUOOXI', rating=3.0),
 Row(item='B008G6OOHA', user='A1FTZ5LLEX7NCM', rating=4.0),
 Row(item='B00A9JNR8E', user='A1R2JUOGIYH6HO', rating=4.0),
 Row(item='B00000JH3R', user='A2X7C89I7YRX1O', rating=5.0),
 Row(item='B000RXPU0U', user='A24BSKCWXC4M6D', rating=5.0),
 Row(item='B00DWXUYN0', user='A3JX1D26WFEXOS', rating=5.0),
 Row(item='B000E9DPVI', user='A2IWHA1XEYSPD6', rating=5.0),
 Row(item='B001EB9F3C', user='A2340917M1HHZ3', rating=4.0)]

Useful function: indexing ID(string) to numerical index

In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline


In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").setHandleInvalid("keep").fit(df_train) for column in list(set(df_train.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexers)
df_ID = pipeline.fit(df_train).transform(df_train)

In [ ]:
df_ID.head(10)

[Row(item='B008H54GVE', user='A2OIMJEGOCTQ87', rating=4.0, item_index=5709.0, user_index=1292.0),
 Row(item='B001NLISDG', user='A38KQZS5M1A8T8', rating=4.0, item_index=1849.0, user_index=4940.0),
 Row(item='B007S3S8HO', user='A1F5O1USOUOOXI', rating=3.0, item_index=937.0, user_index=669.0),
 Row(item='B008G6OOHA', user='A1FTZ5LLEX7NCM', rating=4.0, item_index=10888.0, user_index=107.0),
 Row(item='B00A9JNR8E', user='A1R2JUOGIYH6HO', rating=4.0, item_index=1979.0, user_index=1210.0),
 Row(item='B00000JH3R', user='A2X7C89I7YRX1O', rating=5.0, item_index=979.0, user_index=4764.0),
 Row(item='B000RXPU0U', user='A24BSKCWXC4M6D', rating=5.0, item_index=6385.0, user_index=2866.0),
 Row(item='B00DWXUYN0', user='A3JX1D26WFEXOS', rating=5.0, item_index=9620.0, user_index=8104.0),
 Row(item='B000E9DPVI', user='A2IWHA1XEYSPD6', rating=5.0, item_index=420.0, user_index=11230.0),
 Row(item='B001EB9F3C', user='A2340917M1HHZ3', rating=4.0, item_index=51.0, user_index=10687.0)]

In [ ]:
(train, test) = df_ID.randomSplit([0.8, 0.2], seed = 2022)

Useful function: Construct Binary data \\
adding 1 for watched and 0 for not watched

In [ ]:
def get_binary_data(ratings):
    ratings = df_ID.withColumn('binary', lit(1))
    userIds = df_ID.select("user_index").distinct()
    itemIds = df_ID.select("item_index").distinct()

    user_item = userIds.crossJoin(itemIds).join(ratings, ['user_index', 'item_index'], "left")
    user_item = user_item.select(['user_index', 'item_index', 'binary']).fillna(0)
    return user_item

user_item = get_binary_data(df_ID)

In [ ]:
user_item.show()

+----------+----------+------+
|user_index|item_index|binary|
+----------+----------+------+
|    5776.0|     305.0|     0|
|    5776.0|    2815.0|     0|
|    5776.0|    9753.0|     0|
|    5776.0|     692.0|     0|
|    5776.0|     299.0|     0|
|    5776.0|     596.0|     0|
|    5776.0|     496.0|     0|
|    5776.0|    5776.0|     0|
|    5776.0|    2734.0|     0|
|    5776.0|    1051.0|     0|
|    5776.0|    2862.0|     0|
|    5776.0|     769.0|     0|
|    5776.0|    6433.0|     0|
|    5776.0|    5360.0|     0|
|    5776.0|     934.0|     0|
|    5776.0|    5983.0|     0|
|    5776.0|    6454.0|     0|
|    5776.0|   11757.0|     0|
|    5776.0|     558.0|     0|
|    5776.0|    7313.0|     0|
+----------+----------+------+
only showing top 20 rows



Useful Function: pyspark recommandation system ML model \\
Remember to set coldStartStrategy="nan" to make sure model not ignore null data.

Spark allows users to set the coldStartStrategy parameter to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values. The evaluation metric will then be computed over the non-NaN data and will be valid. Usage of this parameter is illustrated in the example below.

In [ ]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Create ALS model
als = ALS(
         userCol="user_index", 
         itemCol="item_index",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="nan"
)

In [ ]:
# Import the requisite packages
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")


In [ ]:
#Fit cross validator to the 'train' dataset
model = als.fit(train)



Useful function: fill value to NaN part \\
example : \\
a = a.na.fill(value=999)

In [ ]:
# View the predictions
test_predictions = model.transform(test)
# rememver to fix nan in prediction
test_predictions=test_predictions.na.fill()

RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

1.2857136579327189


In [ ]:
df_public = spark.read.csv('',header=True,inferSchema=True)
df_public_ID = pipeline.fit(df_public).transform(df_public)

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(df_public_ID)
# rememver to fix nan in prediction
predictions=predictions.na.fill(value=)

rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

RMSE=1.2798606973227526
+----------+--------------+------+----------+----------+----------+
|      item|          user|rating|item_index|user_index|prediction|
+----------+--------------+------+----------+----------+----------+
|B0006IRTU0|A2CYXQOAR1EJRQ|     5|     471.0|   18105.0|      4.35|
|B000TK8440|A35B72PSA30R67|     5|     183.0|    1699.0|  4.160066|
|B0002BSTY6|A194UXXKM11698|     4|    7919.0|    1160.0| 3.5690026|
|B006HCVT5A|A3ISFBZ5UFK81I|     5|    3045.0|   16376.0| 5.3729506|
|B004R1ZUNA| A3Y0IB3VYLD6A|     5|     297.0|    5296.0|  3.933884|
|B00428LJ06|A1MLBMJSFK6BIJ|     5|     142.0|    6338.0|  3.897945|
|B004GXIDYM|A34BONVNM07TRG|     5|    2537.0|   18436.0|      4.35|
|B0040GK7NK| AR3EVUQF0AC7R|     5|    1901.0|    1433.0|  3.746513|
|B00A88EPCI|A3L249C56OJI7D|     5|     838.0|    8140.0|  4.769078|
|B000IBPD76| ARSNAGZWXP7GN|     5|      39.0|   13708.0| 3.7322226|
|B0063NC3N0|A3AZPAZXGOD4VL|     5|    1495.0|   16190.0| 2.6549904|
|B001Q1A2P0|A168O2YKPE9B

In [ ]:
df_testall = spark.read.csv('/content/drive/MyDrive/TA/2022-CCBDA/HW5/test_all.csv',header=True,inferSchema=True)


In [ ]:
df_test_ID = pipeline.fit(df_testall).transform(df_testall)

In [ ]:
df_test_ID.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- item_index: double (nullable = false)
 |-- user_index: double (nullable = false)



In [ ]:
df_test_ID.describe().show()

+-------+--------------------+--------------------+------------------+-----------------+
|summary|                item|                user|        item_index|       user_index|
+-------+--------------------+--------------------+------------------+-----------------+
|  count|               83799|               83799|             83799|            83799|
|   mean|2.1813556526264625E9|                null|3844.3237150801324|8276.219012160049|
| stddev|2.1882518602245975E9|                null| 3647.800641218535|6463.700444398778|
|    min|          0439893577|A012468118FTQAINE...|               0.0|              0.0|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|           11824.0|          19127.0|
+-------+--------------------+--------------------+------------------+-----------------+



In [ ]:
df_test_ID.show()

+----------+--------------+----------+----------+
|      item|          user|item_index|user_index|
+----------+--------------+----------+----------+
|B0015FRC32|A28QKOPBDPSHE5|   10076.0|   10890.0|
|B000EUKRY0|A1PTTEYFE49BQM|     104.0|    6406.0|
|B007XPLI56|A2C5VTBNC6I5MY|     404.0|   15278.0|
|B003A5RTHO|A12IOCD2A7OC7K|    6845.0|    2525.0|
|B006HCVT5A|A3ISFBZ5UFK81I|    3045.0|   16376.0|
|B004GXIDYM|A34BONVNM07TRG|    2537.0|   18436.0|
|B004R1ZUNA| A3Y0IB3VYLD6A|     297.0|    5296.0|
|B0006IRTU0|A2CYXQOAR1EJRQ|     471.0|   18105.0|
|B00A88EPCI|A3L249C56OJI7D|     838.0|    8140.0|
|B00428LJ06|A1MLBMJSFK6BIJ|     142.0|    6338.0|
|B0002BSTY6|A194UXXKM11698|    7919.0|    1160.0|
|B000IBPD76| ARSNAGZWXP7GN|      39.0|   13708.0|
|B000TK8440|A35B72PSA30R67|     183.0|    1699.0|
|B000EQGT00|A304ILYRZ145SI|     135.0|     710.0|
|B00005YVRN|A3118YKNMNAS33|     134.0|    2209.0|
|B0027FFMBS|A3LHE5MHDF7X2R|   11442.0|   12545.0|
|B00AAPHZVW| ANOST6C92T7HB|     557.0|   18970.0|


In [ ]:
testall_predictions=model.transform(df_test_ID)

In [ ]:
testall_predictions.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- item_index: double (nullable = false)
 |-- user_index: double (nullable = false)
 |-- prediction: float (nullable = false)



In [ ]:
testall_predictions.describe().show()

+-------+--------------------+--------------------+------------------+-----------------+----------+
|summary|                item|                user|        item_index|       user_index|prediction|
+-------+--------------------+--------------------+------------------+-----------------+----------+
|  count|               83799|               83799|             83799|            83799|     83799|
|   mean|2.1813556526264625E9|                null|3844.3237150801324|8276.219012160049|       NaN|
| stddev|2.1882518602245984E9|                null|3647.8006412185086|6463.700444398735|       NaN|
|    min|          0439893577|A012468118FTQAINE...|               0.0|              0.0|       0.0|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|           11824.0|          19127.0|       NaN|
+-------+--------------------+--------------------+------------------+-----------------+----------+



In [ ]:
testall_predictions.show()

+----------+--------------+----------+----------+----------+
|      item|          user|item_index|user_index|prediction|
+----------+--------------+----------+----------+----------+
|B0006IRTU0|A2CYXQOAR1EJRQ|     471.0|   18105.0|       NaN|
|B000TK8440|A35B72PSA30R67|     183.0|    1699.0|  4.160066|
|B0002BSTY6|A194UXXKM11698|    7919.0|    1160.0| 3.5690026|
|B006HCVT5A|A3ISFBZ5UFK81I|    3045.0|   16376.0| 5.3729506|
|B004R1ZUNA| A3Y0IB3VYLD6A|     297.0|    5296.0|  3.933884|
|B00428LJ06|A1MLBMJSFK6BIJ|     142.0|    6338.0|  3.897945|
|B004GXIDYM|A34BONVNM07TRG|    2537.0|   18436.0|       NaN|
|B0040GK7NK| AR3EVUQF0AC7R|    1901.0|    1433.0|  3.746513|
|B00A88EPCI|A3L249C56OJI7D|     838.0|    8140.0|  4.769078|
|B000IBPD76| ARSNAGZWXP7GN|      39.0|   13708.0| 3.7322226|
|B0063NC3N0|A3AZPAZXGOD4VL|    1495.0|   16190.0| 2.6549904|
|B001Q1A2P0|A168O2YKPE9BE8|   10170.0|      25.0| 4.2140884|
|B007XPLI56|A2C5VTBNC6I5MY|     404.0|   15278.0|  2.810014|
|B003A5RTHO|A12IOCD2A7OC

Useful function: concate User Item to generate U_I column

In [ ]:
from pyspark.sql.functions import concat, col, lit

out = testall_predictions.select(concat(col("user"), lit("_"), col("item")) , testall_predictions.prediction)

In [ ]:
out.printSchema()

root
 |-- concat(user, _, item): string (nullable = true)
 |-- prediction: float (nullable = false)



In [ ]:
out.show()

+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A2CYXQOAR1EJRQ_B0...|       NaN|
| A35B72PSA30R67_B0...|  4.160066|
| A194UXXKM11698_B0...| 3.5690026|
| A3ISFBZ5UFK81I_B0...| 5.3729506|
| A3Y0IB3VYLD6A_B00...|  3.933884|
| A1MLBMJSFK6BIJ_B0...|  3.897945|
| A34BONVNM07TRG_B0...|       NaN|
| AR3EVUQF0AC7R_B00...|  3.746513|
| A3L249C56OJI7D_B0...|  4.769078|
| ARSNAGZWXP7GN_B00...| 3.7322226|
| A3AZPAZXGOD4VL_B0...| 2.6549904|
| A168O2YKPE9BE8_B0...| 4.2140884|
| A2C5VTBNC6I5MY_B0...|  2.810014|
| A12IOCD2A7OC7K_B0...| 5.1152215|
| A304ILYRZ145SI_B0...|  3.798786|
| ANOST6C92T7HB_B00...| 0.7698014|
| A3J1CEZ30ZOJ7S_B0...|       NaN|
| A28QKOPBDPSHE5_B0...|  4.533288|
| A1PTTEYFE49BQM_B0...|  4.591045|
| A3118YKNMNAS33_B0...|  3.016202|
+---------------------+----------+
only showing top 20 rows



Remember to fix NaN value

In [ ]:
out.show()

+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A2CYXQOAR1EJRQ_B0...|      4.35|
| A35B72PSA30R67_B0...|  4.160066|
| A194UXXKM11698_B0...| 3.5690026|
| A3ISFBZ5UFK81I_B0...| 5.3729506|
| A3Y0IB3VYLD6A_B00...|  3.933884|
| A1MLBMJSFK6BIJ_B0...|  3.897945|
| A34BONVNM07TRG_B0...|      4.35|
| AR3EVUQF0AC7R_B00...|  3.746513|
| A3L249C56OJI7D_B0...|  4.769078|
| ARSNAGZWXP7GN_B00...| 3.7322226|
| A3AZPAZXGOD4VL_B0...| 2.6549904|
| A168O2YKPE9BE8_B0...| 4.2140884|
| A2C5VTBNC6I5MY_B0...|  2.810014|
| A12IOCD2A7OC7K_B0...| 5.1152215|
| A304ILYRZ145SI_B0...|  3.798786|
| ANOST6C92T7HB_B00...| 0.7698014|
| A3J1CEZ30ZOJ7S_B0...|      4.35|
| A28QKOPBDPSHE5_B0...|  4.533288|
| A1PTTEYFE49BQM_B0...|  4.591045|
| A3118YKNMNAS33_B0...|  3.016202|
+---------------------+----------+
only showing top 20 rows



Remember to sort by U_I before output

In [ ]:
out.orderBy("concat(user, _, item)").show()

+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A012468118FTQAINE...| 5.5311413|
| A012468118FTQAINE...| 4.6938205|
| A012468118FTQAINE...|  4.176839|
| A0182108CPDLPRCXQ...| 3.4932892|
| A0182108CPDLPRCXQ...| 4.3686213|
| A0182108CPDLPRCXQ...| 1.7171348|
| A026961431MGW0616...|  2.473917|
| A026961431MGW0616...| 3.9460294|
| A026961431MGW0616...| 4.9697084|
| A034597326Z83X79S...|  4.803519|
| A034597326Z83X79S...| 4.0378923|
| A034597326Z83X79S...| 3.7165554|
| A034597326Z83X79S...| 4.9082375|
| A04295422T2ZG087R...|  2.852811|
| A04295422T2ZG087R...| 2.4880018|
| A060131923OZAPX4N...| 3.3063307|
| A060131923OZAPX4N...| 3.7409852|
| A060131923OZAPX4N...| 3.9551451|
| A060131923OZAPX4N...|  4.144915|
| A060131923OZAPX4N...| 3.1489997|
+---------------------+----------+
only showing top 20 rows



In [ ]:
out.orderBy("concat(user, _, item)").coalesce(1).write.format("csv").mode('overwrite').save("/content/drive/MyDrive/TA/2022-CCBDA/HW5/sample_submission.csv")